In [1]:
# prompt = "In this serene bathroom, a pristine white bathtub takes center stage, positioned directly "\
# "beneath a mirror that stretches across the wall. The room is clad in gleaming white tiles, which reflect the soft light and "\
# "create a sense of calm. A glass shower door adorns the bathtub, adding a touch of modernity to the space. A few plush towels "\
# "are carefully laid out nearby, with one draped casually over the side of the tub, ready for use. To complete the tranquil "\
# "atmosphere, a mirror stands adjacent to the tub, its reflective surface bouncing light and creating the illusion of a more "\
# "spacious area."
prompt = "A vibrant and bustling urban setting, with a tall, green clock standing prominently on the sidewalk. The clock is surrounded by a variety of people, including two men conversing on the sidewalk, one wearing a red shirt and the other a grey hoodie. The men are standing on a brick-paved sidewalk, lined with trees that have sparse foliage. In the background, a white truck is parked on the curb, with bicycles chained to a pole nearby. A parking meter stands on the sidewalk, with a yellow top and a few coins inserted. To the left of the clock, a white work truck is parked, with a red vehicle parked behind it. A brick building with a white canopy stands nearby, with a black sign above the entrance. The building's exterior is made of white bricks, with a few windows and a door. A tall, metal light post stands nearby, casting a warm glow over the scene. The atmosphere is lively, with the sound of people chatting and the hum of traffic in the background. The sun is shining brightly, casting long shadows across the sidewalk and buildings. The overall impression is one of a thriving urban community, with a mix of old and new architecture and a sense of energy and activity."

In [ ]:
import torch
from diffusers import PixArtSigmaPipeline
from transformers import T5Tokenizer
import os

parent_path = os.getcwd()
cache_dir=os.path.join(parent_path, "cache")

torch.set_float32_matmul_precision("high")
torch._inductor.config.conv_1x1_as_mm = True
torch._inductor.config.coordinate_descent_tuning = True
torch._inductor.config.epilogue_fusion = False
torch._inductor.config.coordinate_descent_check_all_directions = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# You can replace the checkpoint id with "PixArt-alpha/PixArt-Sigma-XL-2-512-MS" too.
tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-11b", legacy=False)
repo_id = "PixArt-alpha/PixArt-Sigma-XL-2-1024-MS"
pipe = PixArtSigmaPipeline.from_pretrained(
       repo_id, tokenizer=tokenizer,
       torch_dtype=torch.float16, 
       cache_dir=cache_dir, 
       use_safetensors=True
       ).to(device)
# Enable memory optimizations.
# pipe.enable_model_cpu_offload()

pipe.set_progress_bar_config(disable=True)

pipe.transformer.to(memory_format=torch.channels_last)
pipe.vae.to(memory_format=torch.channels_last)

pipe.transformer = torch.compile(pipe.transformer, mode="max-autotune", fullgraph=True)
pipe.vae.decode = torch.compile(pipe.vae.decode, mode="max-autotune", fullgraph=True)

# beautifulsoup4 and ftfy packages

In [5]:
# prompt = "A small cactus with a happy face in the Sahara desert."   max_sequence_length = 300 default
img_width = 512
img_height = 512 
image = pipe(prompt,
             num_inference_steps=30,
             guidance_scale=8.0, width=img_width, height=img_height
            ).images[0]

In [ ]:
import Generator as gen
import torch
import os

parent_path = os.getcwd()
cache_dir=os.path.join(parent_path, "cache")
print(cache_dir)
repo_id = "PixArt-alpha/PixArt-Sigma-XL-2-1024-MS"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

pipe = gen.get_pixart_sigma_pipeline(repo_id, cache_dir, device, torch_compile=False)

In [3]:
num_inference_steps = 30
img_width = 512
img_height = 512
prompts = [prompt, prompt]
images = gen.run_pixart_sigma_t2i(pipe, num_inference_steps, img_width, img_height, prompts, manual_seed=False, seed=None)

In [ ]:
images[1]